# Universidad del Rosario
# Procesamiento de Lenguaje Natural
# Caso de estudio 2 corte 2
# Fabián Sánchez

# Caso corte 2

- Para entregar antes del día martes 18 de abril a las 18:00. En grupos de dos integrantes

- Cargar el .pynb en la carpeta de las aulas virtuales

- Los avances del caso se realizará el día 12 de abril en clase y se evaluarán los avances y el trabajo en la clase.

- El trabajo completo se cargará de acuerdo a las anteriores instrucciones.

Considere el siguiente conjunto de datos

In [1]:
url= "https://raw.githubusercontent.com/Fabian830348/Bases_Datos/master/base_LDA.json"

## Librerias

In [69]:
import pandas as pd
from langdetect import detect
import spacy
from gensim.models import LsiModel
from gensim.models import LdaModel
from gensim import corpora
from collections import defaultdict
import numpy as np
import re

In [3]:
#Este es utilizado para español
nlp_es = spacy.load('es_core_news_sm')
# Este es utilizado oara ingles
nlp_en = spacy.load("en_core_web_sm")

# PUNTO 1

* Considere el siguiente conjunto de documentos ***obtener los temas (Topics) de una serie de artículos (noticias)*** que está etiquetados con un tema (el corpus tiene 20 temas).

- El fichero en formato .Json contiene una serie de documento en los que le asigna una temática.

- Cada elemento del Json contiene:

    - **content**: Contenido del artículo
    - **target**: Identificador del target
    - **target_names**: Nombre del target


* ***NOTA***: *Ya que vamos a realizar un ejercicio de aprendizjae no supervisado, no debemos tener en cuentas los temas en los que alguien (un humano experto) ha clasificado estos textos. La idea del ejercicio es obtener los temas distintos de los que hablan los artículos (***Clusterizar artículos***). En este sentido es útil saber a priori el número de temas distintos que puede tener, pero en ningún caso el target de los artículos entraria al algoritmo de aprendizaje*

* Para realizar este ejercicio se seguirán los siguientes pasos


6. Creacción del modelo

    6.1 Crear un modelo con 20 temas

    6.2 Utilice la instrucción "chunksize=100". ¿Qué efecto tiene?

7. Visualización. 

    7.1 Visualizar los resultados con la librería de *pyLDAvis*

    7.2 ¿Qué puede concluir de la visualización?

8. ¿Qué sucede si seleccionó 12, 15, 17 temas? Haga la visualización en estos casos. ¿Cuál es mejor?

### 1.Cargas Datos

In [5]:
df = pd.read_json(url)
df.head(5)

,content,target,target_names
2362,From: brown@ftms.UUCP (Vidiot)\nSubject: Re: p...,5,comp.windows.x
6854,From: markm@bigfoot.sps.mot.com (Mark Monninge...,7,rec.autos
11123,From: fmsalvat@eos.ncsu.edu (FRANK MICHAE SALV...,10,rec.sport.hockey
7235,From: erics@netcom.com (Eric Smith)\nSubject: ...,9,rec.sport.baseball
10824,From: avg@rodan.UU.NET (Vadim Antonov)\nSubjec...,11,sci.crypt


In [6]:
df.content.iloc[1]

"From: markm@bigfoot.sps.mot.com (Mark Monninger)\nSubject: Re: Auto air conditioning without Freon\nNntp-Posting-Host: 223.250.10.7\nReply-To: rapw20@email.sps.mot.com\nOrganization: SPS\nDistribution: usa\nLines: 20\n\nIn article <1993Apr15.222600.11690@research.nj.nec.com>  \nbehanna@syl.nj.nec.com (Chris BeHanna) writes:\n>  ...\n> \tSeveral chemists already have come up with several substitutes for\n> R12.  You don't hear about them because the Mobile Air Conditioning  \nSociety\n> (MACS), that is, the people who stand to rake in that $300 to $1000 per\n> retrofit per automobile, have mounted an organized campaign to squash  \nthose\n> R12 substitutes out of existence if not ban them altogether (on very  \nshaky\n> technical grounds, at best, on outright lies at worst).\n>  ...\n\nNow, I'm not saying you're wrong because I know that the R-12 substitutes  \nexist, but this sounds a lot like the 200mpg carbs that the oil companies  \nkeep us all from getting.\n\nMark\n\n"

2. Exploración de los datos y entendimiento del conjunto de datos

3. Interpretación del siguiente codigo
```
df.groupby(['target', 'target_names']).count()
```

**El siguiente codigo nos agrupa por la etiqueta y nos dice cuantas noticias hay por cada una de las etiquetas realizando un conteo.**

In [10]:
# Este codigo nos indica cuantas noticias hay en cada una de las etiquetas.
df.groupby(['target', 'target_names']).count()

,,content
target,target_names,
0,alt.atheism,92
1,comp.graphics,120
2,comp.os.ms-windows.misc,122
3,comp.sys.ibm.pc.hardware,138
4,comp.sys.mac.hardware,122
5,comp.windows.x,135
6,misc.forsale,145
7,rec.autos,137
8,rec.motorcycles,118


### 4. Para la Normalización utilizar ***spaCy*** y realizar las siguientes acciones en una sola función

In [36]:
def preprocessing(text):
    #Identificar el idioma
    idioma = detect(text)
    #4.1. Pasar las frases a minúsculas.
    text = text.lower()
    #4.6. Eliminar los emails  \S*@\S*\s
    #\S Identifica caracteres que no son espacio
    #* Lo realiza continuamente
    #\s Identifica espacios
    text = re.sub("\S*@\S*\s","",text)
    #print(text)
    #4.2. Eliminar los signos de puntuación.
    #4.7. Eliminar los saltos de línea
    #4.8. Eliminar las comillas simples
    text = re.sub("[^A-Za-z0-9]+","", text) 
    # Tokenización
    if idioma == "en":
        doc = nlp_en(text)
    else:
        doc = nlp_es(text)
    #Realizamos una lista donde tenemos las palabras tokenizadas, su lema, su longitud
    # su tag y si son o no palabras de parada.
    resultado = [[tk.text, len(tk.text),tk.lemma_, tk.pos_, tk.is_alpha, tk.is_stop] for tk in doc]
    # 4.5. Eliminar las Stop-Words.
    resultado = [lista for lista in resultado if lista[5] == False]
    #4.3. Eliminar las palabras con menos de 3 carácteres.
    #4.4. Eliminar las palabras con mas de 12 carácteres.
    resultado = [lista for lista in resultado if (lista[1] <12) and (lista[1] > 3)]
    # 4.9. Filtrar las palabras que no sean Sustantivo, Adjetivo, Verbo o Adverbio.
    resultado = [lista for lista in resultado if (lista[3] == 'NOUN') or (lista[3] == 'ADJ') or (lista[3] == 'ADV') or (lista[3] == 'VERB')]
    #4.10. Pasar la palabra a su lema
    #Tomamos los lemas de cada una de las palabras de la lista despues de la limpieza.
    result = []
    for i in resultado:
        result.append(i[2])
        
    return result


In [48]:
df['pre-procesado'] = df['content'].apply(lambda texto: preprocessing(texto))
df

,content,target,target_names,pre-procesado
2362,From: brown@ftms.UUCP (Vidiot)\nSubject: Re: p...,5,comp.windows.x,"[vidiot, subject, problem, xvertext, package, ..."
6854,From: markm@bigfoot.sps.mot.com (Mark Monninge...,7,rec.autos,"[subject, auto, freon, nntp, post, host, reply..."
11123,From: fmsalvat@eos.ncsu.edu (FRANK MICHAE SALV...,10,rec.sport.hockey,"[michae, subject, division, correction, origin..."
7235,From: erics@netcom.com (Eric Smith)\nSubject: ...,9,rec.sport.baseball,"[phillie, keyword, phillie, line, service, gue..."
10824,From: avg@rodan.UU.NET (Vadim Antonov)\nSubjec...,11,sci.crypt,"[subject, invent, crypto, policy, statement, f..."
...,...,...,...,...
9595,From: asket@acad2.alaska.edu\nSubject: When is...,15,soc.religion.christian,"[subject, couple, married, line, marriage, cou..."
9554,From: dudek@acsu.buffalo.edu (The Cybard)\nSub...,2,comp.os.ms-windows.misc,"[cybard, subject, midi, file, win3, summary, p..."
5969,From: HoffmanE@space1.spacenet.jhuapl.edu (Hof...,14,sci.space,"[subject, drag, free, satellite, utexas, mail,..."
8239,From: katkere@krusty.eecs.umich.edu (Arun Katk...,1,comp.graphics,"[subject, cylinder, reply, line, article, writ..."


### 5. Crear del diccionario y la bolsa de palabras
5.1 Corpus tokenizado: "*documents_tok*"

In [147]:
x = df["pre-procesado"]

In [148]:

# Creamos el diccionario (vocabulario)
frequency = defaultdict(int)
for doc in x:
    for token in doc:
        frequency[token] += 1

print(frequency)



defaultdict(<class 'int'>, {'vidiot': 2, 'subject': 1815, 'problem': 780, 'xvertext': 2, 'package': 147, 'reply': 590, 'hangout': 1, 'line': 2898, 'article': 1702, 'write': 2251, 'need': 716, 'future': 98, 'display': 162, 'rotate': 19, 'text': 124, 'notice': 87, 'look': 739, 'good': 1117, 'slight': 15, 'assume': 172, 'font': 36, 'load': 90, 'shouldn': 39, 'difficult': 69, 'split': 35, 'function': 137, 'interface': 102, 'routine': 54, 'xfontstruct': 1, 'think': 1268, 'author': 123, 'case': 467, 'plan': 148, 'email': 298, 'address': 252, 'bounce': 29, 'current': 172, 'maintainer': 2, 'mail': 409, 'mention': 199, 'know': 1504, 'brit': 5, 'backwards': 5, 'real': 254, 'state': 430, 'brown': 18, 'rutger': 18, 'internet': 235, 'auto': 86, 'freon': 2, 'nntp': 986, 'post': 1557, 'host': 1128, 'chemist': 6, 'come': 712, 'substitute': 13, 'hear': 372, 'mobile': 8, 'society': 116, 'mac': 17, 'people': 1326, 'stand': 134, 'rake': 3, 'retrofit': 4, 'automobile': 28, 'mount': 25, 'organize': 34, 'cam

 5.2 Diccionario: "*dictionary*"

In [149]:
documents = [[token for token in doc] for doc in x]
dictionary = corpora.Dictionary(documents)
print('Diccionario:')
print(dictionary.token2id)

Diccionario:
{'address': 0, 'article': 1, 'assume': 2, 'author': 3, 'backwards': 4, 'bounce': 5, 'brit': 6, 'brown': 7, 'case': 8, 'current': 9, 'difficult': 10, 'display': 11, 'email': 12, 'font': 13, 'function': 14, 'future': 15, 'good': 16, 'hangout': 17, 'interface': 18, 'internet': 19, 'know': 20, 'line': 21, 'load': 22, 'look': 23, 'mail': 24, 'maintainer': 25, 'mention': 26, 'need': 27, 'notice': 28, 'package': 29, 'plan': 30, 'problem': 31, 'real': 32, 'reply': 33, 'rotate': 34, 'routine': 35, 'rutger': 36, 'shouldn': 37, 'slight': 38, 'split': 39, 'state': 40, 'subject': 41, 'text': 42, 'think': 43, 'vidiot': 44, 'write': 45, 'xfontstruct': 46, 'xvertext': 47, 'altogether': 48, 'auto': 49, 'automobile': 50, 'bad': 51, 'campaign': 52, 'carb': 53, 'chemist': 54, 'come': 55, 'company': 56, 'exist': 57, 'existence': 58, 'freon': 59, 'get': 60, 'ground': 61, 'hear': 62, 'host': 63, 'lie': 64, 'mac': 65, 'mark': 66, 'mobile': 67, 'mount': 68, 'nntp': 69, 'organize': 70, 'outright': 

5.3 Corpus: "*corpus*' que es la bolsa de palabras de gensim

In [150]:
# Creamos la Bolsa de Palabras (Indice de la palabra, frecuencia)
corpus = [dictionary.doc2bow(doc) for doc in x]
print('\nBolsa de Palabras:')
print(corpus)


Bolsa de Palabras:
[[(0, 3), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 3), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 2), (30, 1), (31, 2), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 2), (45, 1), (46, 1), (47, 2)], [(1, 1), (16, 1), (20, 1), (21, 1), (33, 1), (41, 1), (45, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 3), (83, 1), (84, 1)], [(1, 1), (21, 1), (33, 1), (41, 1), (45, 1), (60, 1), (85, 2), (86, 1), (87, 1), (88, 2), (89, 3), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 

### 6. Creacción del modelo

**Modelo LDA**

In [155]:
lda_model = LdaModel(corpus=corpus, 
                     num_topics=10, 
                     id2word=dictionary, 
                     random_state=5,
                     chunksize=100)

**Matriz de probabilidades - Matriz U** 

In [154]:
docs_topics = np.array([[tup[1] for tup in lst] for lst in lda_model[corpus]])
docs_topics


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2500,) + inhomogeneous part.

# PUNTO 2

- Con los datos normalizados y vectorizados aplique el método LSI. Compare los resultados.

**Model LSI**

In [156]:
lsi_model = LsiModel(corpus=corpus, num_topics=20, id2word=dictionary)

**Matriz U**

In [157]:
U_matrix = lsi_model.projection.u
U_matrix

array([[ 8.11123242e-02,  1.06277915e-01,  9.07353770e-02, ...,
         1.81222564e-02, -3.85089556e-02,  5.44490085e-02],
       [ 6.00683358e-02, -8.71309992e-03,  1.07683780e-02, ...,
         1.82443425e-01, -4.80599197e-02, -1.41615259e-02],
       [ 1.40878086e-02,  5.21468523e-03, -1.37944636e-02, ...,
        -3.18040512e-02, -2.26041778e-03,  1.89967081e-03],
       ...,
       [ 6.01209770e-05, -1.32142329e-05,  7.43075372e-06, ...,
         8.92579861e-04, -5.04787457e-05, -1.14426238e-04],
       [ 1.50302443e-05, -3.30355822e-06,  1.85768843e-06, ...,
         2.23144965e-04, -1.26196864e-05, -2.86065596e-05],
       [ 4.50907328e-05, -9.91067466e-06,  5.57306529e-06, ...,
         6.69434896e-04, -3.78590593e-05, -8.58196787e-05]])

In [158]:
pd.DataFrame(U_matrix, columns=['Topic 1', 'Topic 2', 'Topic 3','Topic 4', 'Topic 5', 'Topic 6',
                                'Topic 7', 'Topic 8', 'Topic 9','Topic 10', 'Topic 11', 'Topic 12',
                                'Topic 13', 'Topic 14', 'Topic 15','Topic 16', 'Topic 17', 'Topic 18',
                                'Topic 19', 'Topic 20'], 
             index=dictionary.token2id.keys()).head(15)

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10,Topic 11,Topic 12,Topic 13,Topic 14,Topic 15,Topic 16,Topic 17,Topic 18,Topic 19,Topic 20
address,0.081112,0.106278,0.090735,-0.104368,-0.072858,-0.031498,0.029353,0.015009,-0.002236,-0.002388,-0.004037,-0.001869,-0.011803,-0.000315,-0.013137,0.009504,-0.014403,0.018122,-0.038509,0.054449
article,0.060068,-0.008713,0.010768,0.070231,0.025212,-0.097689,-0.170417,-0.072963,-0.161889,0.093115,0.019409,-0.127954,-0.023782,-0.041509,-0.001826,0.046641,0.037049,0.182443,-0.048060,-0.014162
assume,0.014088,0.005215,-0.013794,0.007474,0.003392,-0.019387,-0.014671,-0.006691,-0.020324,0.017246,0.006234,0.004001,0.002790,-0.008268,0.015033,-0.000705,0.004747,-0.031804,-0.002260,0.001900
author,0.035019,0.039759,-0.052201,-0.012897,-0.010098,-0.019771,-0.014251,-0.060741,-0.010564,0.092005,-0.037727,0.175965,-0.037402,-0.000842,-0.005085,0.006794,0.011947,0.021053,0.022413,0.001814
backwards,0.001296,0.001077,0.000024,0.002546,0.003091,0.002449,-0.000627,-0.003029,-0.001661,-0.006504,-0.005951,0.000195,0.001805,0.001153,0.001181,0.001573,0.000279,-0.001025,-0.000956,-0.000267
bounce,0.004747,0.005780,0.008701,-0.005968,-0.003803,-0.003243,0.000975,0.000158,-0.001925,-0.000874,0.001166,-0.001998,-0.003973,0.001932,-0.006555,-0.000553,-0.003175,0.004493,-0.002396,0.000791
brit,0.000108,-0.000052,0.000021,0.000077,0.000009,-0.000183,-0.000297,0.000046,-0.000363,0.000128,0.000235,-0.000178,-0.000067,-0.000017,0.000057,-0.000003,0.000300,0.000320,-0.000205,-0.000395
brown,0.000710,-0.000282,0.000095,0.000564,-0.000004,-0.000545,-0.001568,-0.000515,-0.000671,0.000822,-0.000087,-0.000425,0.000698,0.000238,-0.001160,-0.000058,-0.000738,0.001307,0.000430,0.001367
case,0.069076,0.048831,0.013557,-0.000628,0.010509,-0.048497,-0.010518,-0.030886,-0.063627,-0.015111,-0.021847,0.024218,0.031144,-0.028671,0.020531,-0.145253,-0.024533,-0.060819,-0.012446,0.024712
current,0.029994,0.021153,-0.008934,0.004685,0.015708,0.028763,-0.017975,0.003752,-0.022841,0.000362,-0.040742,-0.013051,0.003812,-0.031506,-0.000012,-0.208273,-0.036639,-0.006427,-0.021946,0.013012


# PUNTO 3

- Con los datos normalizados y vectorizados. ¿Es posible aplicar k-means?

- En caso de una respuesta afirmativa. Utilice alguno el método del codo, el método de la silueta, y el método GAP para determinar cuáles valores de $k$ (número de clusters) son sugeridos y aplique el modelo.

- Comparar resultados con los métodos anteriores: k-means, LDA y LSI.

- Concluya